In [ ]:
import redis
import pickle
import time
import uuid
import pandas as pd
from datetime import datetime, timedelta
from typing import List, Dict, Optional

class MinuteDataPublisher:
    def __init__(self, config_path: str = "redis.conf"):
        """初始化发布器，完全参考RemoteSender的初始化逻辑"""
        self.redis_config = self._load_redis_config(config_path)
        self.redis = redis.Redis(
            host=self.redis_config["host"],
            port=self.redis_config["port"],
            password=self.redis_config["password"],
            decode_responses=False  # 保持二进制传输，与参考代码一致
        )
        self.task_queue = "function_calls"  # 与参考代码队列名称一致
        self._test_redis_connection()
        print(f"✅ 发布器初始化完成 | pandas版本: {pd.__version__}")

    def _load_redis_config(self, config_path: str) -> Dict[str, str]:
        """加载Redis配置，完全复用参考代码的解析逻辑"""
        config = {"host": "localhost", "port": 6379, "password": ""}
        try:
            with open(config_path, "r", encoding="utf-8") as f:
                for line in f:
                    line = line.strip()
                    if line.startswith("host="):
                        config["host"] = line.split("=", 1)[1].strip()
                    elif line.startswith("port="):
                        config["port"] = int(line.split("=", 1)[1].strip())
                    elif line.startswith("password="):
                        config["password"] = line.split("=", 1)[1].strip()
            return config
        except Exception as e:
            print(f"⚠️ 配置文件读取失败，使用默认配置: {e}")
            return config

    def _test_redis_connection(self):
        """测试Redis连接，参考RemoteSender的_test_connection方法"""
        try:
            self.redis.ping()
            print(f"✅ Redis连接成功 | {self.redis_config['host']}:{self.redis_config['port']}")
        except Exception as e:
            print(f"❌ Redis连接失败: {e}")
            raise SystemExit(1)

    def _get_valid_stocks(self, daily_data_path: str) -> List[str]:
        """获取有效股票列表，参考日K数据筛选逻辑"""
        try:
            daily_df = pd.read_parquet(daily_data_path)
            # 筛选未停牌股票（paused=0）并去重
            valid_stocks = daily_df[daily_df["paused"] == 0]["stock_code"].unique().tolist()
            print(f"📊 有效股票数量: {len(valid_stocks)}")
            return valid_stocks
        except Exception as e:
            print(f"⚠️ 读取股票列表失败，使用默认列表: {e}")
            return []  # 默认股票

    def publish_tasks(
        self,
        func_name: str = "fetch_minute_stock_data",
        daily_data_path: str = r"D:\workspace\xiaoyao\data\stock_daily_price.parquet",
        batch_size: int = 1,
        max_queue_len: int = 10000
    ):
        """发布任务的核心方法，完全参考RemoteSender.call_remote_function的任务结构"""
        # 1. 准备基础数据

        valid_stocks = self._get_valid_stocks(daily_data_path)
        if not valid_stocks:
            print("❌ 无有效股票，退出发布")
            return

        # 2. 按股票批次发布任务（每个任务包含完整日期范围）
        total_batches = (len(valid_stocks) + batch_size - 1) // batch_size
        start_date = '20250101'
        # end_date 取昨日
        end_date = (datetime.now() - timedelta(days=1)).strftime('%Y%m%d')
        
        print(f"🚀 开始发布任务 | 总批次: {total_batches} | 日期范围: {start_date}-{end_date}")

        for batch_idx in range(total_batches):
            # 计算当前批次股票
            start = batch_idx * batch_size
            end = min(start + batch_size, len(valid_stocks))
            batch_stocks = valid_stocks[start:end]
            
            # 生成符合参考代码格式的任务ID
            task_id = f"task_{uuid.uuid4().hex[:8]}"
            
            # 控制队列长度，避免溢出（参考代码的流量控制逻辑）
            while self.redis.llen(self.task_queue) >= max_queue_len:
                print(f"⏳ 队列已满（{max_queue_len}），等待30秒...")
                time.sleep(30)

            # 构建任务结构：完全参考RemoteSender的task格式
            task = {
                "func_name": func_name,          # 函数名
                "args": (start_date, end_date, batch_stocks),  # 位置参数（与函数参数对应）
                "kwargs": {},                    # 关键字参数（必须包含，即使为空）
                "task_id": task_id               # 任务ID
            }

            # 序列化任务：使用与参考代码相同的pickle.dumps
            self.redis.rpush(self.task_queue, pickle.dumps(task))
            
            # 记录任务元信息（便于结果处理）
            self.redis.hset(
                "task_metadata",
                task_id,
                pickle.dumps({
                    "batch": batch_idx,
                    "stocks_count": len(batch_stocks),
                    "date_range": f"{start_date}-{end_date}"
                })
            )

            # 打印进度（参考代码的简洁输出风格）
            progress = (batch_idx + 1) / total_batches * 100
            print(f"📤 批次 {batch_idx+1}/{total_batches} ({progress:.1f}%) | 任务ID: {task_id[:8]}...")

            # 控制发布速度（低频任务特性）
            if batch_idx < total_batches - 1:
                time.sleep(5)

        print(f"✅ 所有任务发布完成 | 总任务数: {total_batches}")

if __name__ == "__main__":
    # 执行入口：与参考代码的main函数风格一致
    try:
        publisher = MinuteDataPublisher(config_path="redis.conf")
        publisher.publish_tasks(
            func_name="fetch_minute_stock_data",
            daily_data_path=r"D:\workspace\xiaoyao\data\stock_daily_price.parquet",
            batch_size=1,
            max_queue_len=100
        )
    except Exception as e:
        print(f"❌ 发布器执行失败: {e}")


✅ Redis连接成功 | 220.203.1.124:6379
✅ 发布器初始化完成 | pandas版本: 2.3.2
📊 有效股票数量: 5447
🚀 开始发布任务 | 总批次: 5447 | 日期范围: 20250101-20251014
📤 批次 1/5447 (0.0%) | 任务ID: task_6f8...
📤 批次 2/5447 (0.0%) | 任务ID: task_be4...
📤 批次 3/5447 (0.1%) | 任务ID: task_b90...
📤 批次 4/5447 (0.1%) | 任务ID: task_3e5...
📤 批次 5/5447 (0.1%) | 任务ID: task_ee9...
📤 批次 6/5447 (0.1%) | 任务ID: task_9fb...
📤 批次 7/5447 (0.1%) | 任务ID: task_f5c...
📤 批次 8/5447 (0.1%) | 任务ID: task_45b...
📤 批次 9/5447 (0.2%) | 任务ID: task_db0...
📤 批次 10/5447 (0.2%) | 任务ID: task_d93...
📤 批次 11/5447 (0.2%) | 任务ID: task_ada...
📤 批次 12/5447 (0.2%) | 任务ID: task_2ac...
📤 批次 13/5447 (0.2%) | 任务ID: task_310...
📤 批次 14/5447 (0.3%) | 任务ID: task_6d0...
📤 批次 15/5447 (0.3%) | 任务ID: task_788...
📤 批次 16/5447 (0.3%) | 任务ID: task_a90...
📤 批次 17/5447 (0.3%) | 任务ID: task_7df...
📤 批次 18/5447 (0.3%) | 任务ID: task_403...
📤 批次 19/5447 (0.3%) | 任务ID: task_36b...
📤 批次 20/5447 (0.4%) | 任务ID: task_621...
📤 批次 21/5447 (0.4%) | 任务ID: task_afd...
📤 批次 22/5447 (0.4%) | 任务ID: task_c9e...
📤 批次 